In [1]:
import json
from tqdm import tqdm
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import cv2

In [2]:
# Функция Влада
def img_data_create(data, cat, im_path):
    '''Функция, создающая словарь типа {image_id:{'image_size':...,'polygons':[[...],...,[...]],
    'bboxes':[[...],...,[...]]},...} для категории cat, где image_id - id изображения, image_size - размер изображения,
    polygons - массив координат для построения масок объектов на изображении, bboxes - массив ограничивающих рамок объектов на 
    изображении
    Входные параметры: data - словарь с аннотациями для объектов
                       cat - категория
                       im_path - путь к изображениям
    Выходные данные: словарь'''
    cat_id = [x for x in data['categories'] if x['name'] == cat][0]['id']
    anns = [ann for ann in data['annotations'] if ann['category_id'] == cat_id]
    img_data = {}
    for ann in anns:
        img_data[ann['image_id']] = {}
        img_data[ann['image_id']]['polygons'] = [] 
        img_data[ann['image_id']]['bboxes'] = [] 
    for ann in anns:
        img_data[ann['image_id']]['polygons'].append(ann['segmentation'][0])
        img_data[ann['image_id']]['bboxes'].append([ann['bbox'][0],ann['bbox'][1],ann['bbox'][0]+ann['bbox'][2]-1,ann['bbox'][1]+ann['bbox'][3]-1])
    for im_id in tqdm(img_data):
        im_fn = [x for x in data['images'] if x['id'] == im_id][0]['seg_file_name']
        im = Image.open(f'{im_path}/{im_fn}')
        img_data[im_id]['img_size']=im.size
    return img_data

In [3]:
with open('iSAID_train.json', 'r', encoding='Utf-8') as json_data:
    tmd=json.load(json_data)

categories = list() 
for i in range(0, 15):
    categories.append(tmd['categories'][i]['name'])
print('categories: ', categories)

categories:  ['storage_tank', 'Large_Vehicle', 'Small_Vehicle', 'plane', 'ship', 'Swimming_pool', 'Harbor', 'tennis_court', 'Ground_Track_Field', 'Soccer_ball_field', 'baseball_diamond', 'Bridge', 'basketball_court', 'Roundabout', 'Helicopter']


In [4]:
train_img_data = dict.fromkeys(categories)

im_path = 'train/Semantic_masks/images/images'

for cat in categories: 
    train_img_data[cat] = img_data_create(tmd, cat, im_path)

for cat in categories:
    for key in train_img_data[cat].keys(): 
        train_img_data[cat][key]['seg_file_name'] = 'train/Semantic_masks/images/images/' + tmd['images'][key]['seg_file_name']

100%|██████████| 38/38 [00:00<00:00, 4546.54it/s]


In [6]:
def plot_heatmap(train_img_data, category, target_size=(2400, 2400), save_figure=True):
    """
    Generates a heatmap based on semantic masks and bounding boxes of images for a specified category.

    Parameters:
    - train_img_data: Dictionary containing image data.
    - category: Category for generating the heatmap.
    - target_size: Target size for the images.
    """
    category_data = train_img_data[category]

    filename = 'eda_images/heatmaps/' + category + '_HeatMap.png'

    combined_mask = np.zeros(target_size, dtype=np.float64)

    for image_id, image_info in category_data.items():
        mask_filename = image_info['seg_file_name']
        mask = cv2.imread(mask_filename, cv2.IMREAD_GRAYSCALE).astype(np.float64)

        img_size = mask.shape
        # img_size = image_info['img_size']

        img_resized = cv2.resize(mask, target_size[::-1])
        mask_resized = cv2.resize(mask, target_size[::-1])

        bboxes = image_info['bboxes']

        normalized_bboxes = [
            (bbox[0] * target_size[1] / img_size[1],
             bbox[1] * target_size[0] / img_size[0],
             bbox[2] * target_size[1] / img_size[1],
             bbox[3] * target_size[0] / img_size[0])
            for bbox in bboxes
        ]

        for bbox in normalized_bboxes:
            x1, y1, x2, y2 = bbox
            x1 = int(x1 * target_size[1] / img_size[1])
            y1 = int(y1 * target_size[0] / img_size[0])
            x2 = int(x2 * target_size[1] / img_size[1])
            y2 = int(y2 * target_size[0] / img_size[0])
            combined_mask[y1:y2, x1:x2] += mask_resized[y1:y2, x1:x2]

    plt.figure()

    plt.imshow(combined_mask, cmap='viridis', alpha=1.0, interpolation='sinc')
    plt.title(f"Heatmap for Category: {category}")
    plt.xlabel("Width")
    plt.ylabel("Height")

    cbar = plt.colorbar()
    cbar.set_label('Intensity')

    if save_figure:
        plt.savefig(filename)
        plt.close()
    else:
        plt.show()

In [7]:
image_names = list()
for cat in tqdm(categories): 
    plot_heatmap(train_img_data, cat, save_figure=True)
    image_names.append('eda_images/heatmaps/' + cat + '_HeatMap.png')

  0%|          | 0/15 [00:00<?, ?it/s]

100%|██████████| 15/15 [09:10<00:00, 36.68s/it]


In [8]:
imgages = [cv2.imread(f'{name}') for name in image_names]

image_height, image_width, _ = imgages[0].shape
canvas = np.zeros((5 * image_height, 3 * image_width, 3), dtype=np.uint8)
for i in range(5):
    for j in range(3):
        idx = i * 3 + j
        if idx < len(imgages):
            canvas[i * image_height: (i + 1) * image_height, j * image_width: (j + 1) * image_width, :] = imgages[idx]

cv2.imwrite('eda_images/heatmaps/Total_heat_map.png', canvas)

True